 ## <center> Домашнее задание по курсу УМвАД

### **Задача:** 
* Бинарная классификация (со значениями целевого признака + и -) с помощью "ленивого" алгоритма.
* Тестирование на двух датасетах (SPECT, CARS)



## <font color=green>  #1 Набор данных SPECT

Для выполнения домашней работы по курсу "Упорядоченные множества в анализе данных" был выбран набор данныx Single Proton Emission Computed Tomography (SPECT) heart data. 

Источник: http://archive.ics.uci.edu/ml/machine-learning-databases/spect/

Сводная информация по датасету: 
	* The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images.  
	* Each of the patients is classified into two categories: normal and abnormal. 
	* The database of 267 SPECT image sets (patients) was processed to extract features that summarize the original SPECT images.  
	* As a result, 44 continuous feature pattern was created for each patient.
	* The pattern was further processed to obtain 22 binary feature patterns.
	* The CLIP3 algorithm was used to generate classification rules from these patterns.
	* The CLIP3 algorithm generated rules that were 84.0% accurate (as compared with cardilogists' diagnoses).

Набор данных не содержит пропущенных значений. В обучающей выборке содержится по 40 примеров каждого класса ('0' и '1'). В тестовой выборке - 15 примеров класса '0' и 172 примера класса '1'. 

In [1]:
import numpy

# Загружаем данные из обучающей выборки
data = open("SPECT.train","r")
train = [ i.strip().split(",") for i in data]
plus = [i for i in train if i[0]=="1"]
minus = [i for i in train if i[0]=="0"]
train_joined = [''.join(i) for i in train]
data.close()

In [2]:
print('len train:', len(train))
print('len of one plus:', len(plus[0]), 'overall len plus:', len(plus))
print('len of one minus:', len(plus[0]), 'overall len minus: ', len(minus))

len train: 80
len of one plus: 23 overall len plus: 40
len of one minus: 23 overall len minus:  40


In [3]:
# Загружаем данные из тестовой выборки
data = open("SPECT.test","r")
test = [ i.strip().split(",") for i in data]
test_joined = [''.join(i) for i in test]
data.close()

In [7]:
print('len test: ', len(test_joined))

len test:  187


In [8]:
for i in test_joined:
    print(i)

test1 = ''.join(train[0])
test2 = ''.join(train[1])

11001100011000111001100
11001100001001000000000
10001010010100110000001
10111001010011101000010
10010000100101101000001
10011010010101001000011
11001001111011101000101
11001000011000110000000
10000000100001000000000
11001110011100101101000
11000100011001110001000
11000110001100000001000
10000010100001000010001
11000100011000100011000
11001100011000001001100
11010100101001000010010
11100111101111000100011
11000011001110000100001
11000100000000000000100
11110101101011001000110
11000100001000001010100
11011111111111110010111
10011100111001111010110
11101111000110001100100
11100101001000000000000
11100111001000001000000
11010100100001000100010
11111011101001111001100
11100110001000001001000
10000000000000001000000
11100100111101111001100
11100011001100011001110
10000001100011001000011
11000100100001000000011
10011000110000000000100
10000000100001000000011
10110001100010001000000
10000000000000000101000
11011100111001011000111
10111001110011100000011
10110001100011000000011
1000001100011100

In [12]:
train_bin = []

for i in range(0,len(train)):
    temp = ''.join(train[i][1:])
    train_bin.append(int(temp,2))

#print(len(train_bin), len(train))
plus_bin = train_bin[0:39]
minus_bin = train_bin[40:79]

In [14]:
det_str = train[43][1:]
print(det_str)
to_detect = int(''.join(det_str))
print(to_detect)

['0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
1000000000000000000


In [20]:
# Список голосов за каждый из тренировочных объектов
vote_list = []

# Функция фальсификации гипотез
def falsification(mask_bin, context_bin):
    for i in range(0,len(context_bin)-1):
        if mask_bin & context_bin[i] == mask_bin:
            return 0
        else:
            return 1

In [21]:
# Функция подсчета голосов для неизвестного объекта
def get_vote(to_detect, plus_bin, minus_bin):
    vote_count = 0
    for i in range(0,len(plus)-1):
        mask = to_detect & plus_bin[i]
        vote_count += falsification(mask, minus_bin)
    for i in range(0,len(plus)-1):
        mask = to_detect & minus_bin[i]
        vote_count -= falsification(mask, plus_bin)
    return vote_count

In [22]:
# Для каждого элемента из тренировочного набора считаем, сколько голосов за него
for i in range(0, len(train)):
    det_str = train[i][1:]
    to_detect = int(''.join(det_str))
    vote = get_vote(to_detect, plus_bin, minus_bin)
    vote_list.append(vote)
    #print('vote for object #', i, '('+''.join(det_str)+')', vote)

В наборе данных есть несколько одинаковых объектов, которые классифицируются в обучающей выборка и как "+", и как "-".
Помимо этого, выборка несбалансированная: в отрицательном контексте меньше признаков, поэтому он с большей вероятностью вложится в положительный.
Используя вероятностный подход, можно посмотреть распределение голосов для отрицательных и положительных примеров. 

In [25]:
import numpy as np
print('Среднее (+): ', np.mean(vote_list[0:39]), ', станд. отклонение: ', np.std(vote_list[0:39])/np.sqrt(40))
print('Среднее (-): ', np.mean(vote_list[40:89]), ', станд. отклонение: ', np.std(vote_list[40:89])/np.sqrt(40))

Среднее (+):  13.9487179487 , станд. отклонение:  0.871889135175
Среднее (-):  8.925 , станд. отклонение:  1.41482662365


Для положительных примеров выборочное среднее - 13.9487 и смещено вправо от распределения. Для отрицательных -- среднее 8.925 со смещением влево. В качестве решающего правила можно выбрать количество голосов > 11. Выберем отсечение ближе к среднему положительных голосов, так как дисперсия положительных оказалась на порядок меньше, чем у отрицательных.

In [27]:
# Словарь, в котором хранится количество правильно и неправильно угаданных примеров

cv_res = {
 "11": 0,  #true positive
 "10": 0,  #false negative
 "01": 0,  #false positive
 "00": 0,  #true negative
 "NaN": 0, #unclassified
}

#Основная часть: смотрим на данные из теста и угадываем их.
for i in test_joined:
    det_str = i[1:]
    to_detect = int(''.join(det_str))
    vote = get_vote(to_detect, plus_bin, minus_bin)
    if vote > 11:
        decision = '1'
    else:
        decision = '0'
    if i[0] == '1' and decision == '1':
        cv_res['11'] +=1
    elif i[0] == '1' and decision == '0':
        cv_res['10'] +=1
    elif i[0] == '0' and decision == '1':
        cv_res['01'] +=1
    elif i[0] == '0' and decision == '0':
        cv_res['00'] +=1
    else:
        cv_res['NaN'] +=1

print('Accuracy: ', (cv_res['11']+cv_res['00'])/(cv_res['11']+cv_res['00']+cv_res['01']+cv_res['10']+cv_res['NaN']))
print('True-positive: ', cv_res['11'], 'true-negative: ', cv_res['00'])
print('False-negative: ', cv_res['10'], 'false-positive: ', cv_res['01'])
print('Uncertain:', cv_res['NaN'])

Accuracy:  0.786096256684492
True-positive:  139 true-negative:  8
False-negative:  33 false-positive:  7
Uncertain: 0


По сравнению с CLIP3 алгоритмом, показавшим точность 84% (в описании данных), точность lazy алгоритма классификации 78,6% показывает, что общую структуру в данных он ловит хорошо.

##  <font color=green> #2 Набор данных Cars

Источник: http://archive.ics.uci.edu/ml/machine-learning-databases/car/

Сводная информация по данным:
* Количество объектов: 1728
* Количество признаков: 6

Признаки и значения: 
* **buying (цена)**:       v-high, high, med, low
* **maint (расходы на содержание)**:        v-high, high, med, low
* **doors (количество дверей)**:       2, 3, 4, 5-more
* **persons (вместительность)**:      2, 4, more
* **lug_boot (размер багажника)**:     small, med, big
* **safety (оценка безопасности)**:       low, med, high

In [31]:
# Загружаем данные
data = open("car.data","r")
all_data = [ i.strip().split(",") for i in data]
plus = [i for i in train if i[6]=="acc" or i[6]=="good" or i[6]=="v-good"]
minus = [i for i in train if i[6]=="unacc"]
print('len all data:', len(all_data))

len all data: 1728


In [37]:
# Обрабатываем данные для анализа
# Переведем в бинарный вид

"""   
index    name         values                   replacement values
0        buying       vhigh, high, med, low   11, 10, 01, 00
1        maint        vhigh, high, med, low   11, 10, 01, 00
2        doors        2, 3, 4, 5-more          00, 01, 10, 11
3        persons      2, 4, more               00, 10, 11
4        lug_boot     small, med, big          01, 10, 11
5        safety       low, med, high           01, 10, 11
"""

data = [['','','','','','',''] for x in range(len(all_data))]
replacement_dict = {
 "vhigh": '11',
 "high": '10',
 "med": '01',
 "low": '00',
 "2": '00',
 "3": '01',
 "4": '10',
 "5more": '11',
 "more": '11',
 "small": '01',
 "low": '01',
 "med": '10',
 "big": '11',
 "high": '11',
 "acc": '1',
 "good": '1',
 "vgood": '1',
 "unacc": '0',}

In [38]:
for i in range(0, len(data)):
    for j in range(0, 7):
        data[i][j] = replacement_dict[all_data[i][j]]

# print(train[0])
# print(data[0])

for i in range(0,len(data)):
    data[i] = ''.join(data[i])

In [39]:
from random import shuffle

#Получаем тренировочную и тестовую выборки 

plus = [a for a in data if a[-1]=="1"] # полный положительный контекст
minus = [a for a in data if a[-1]=="0"] # полный отрицательный контекст

plus_shuffle = []
plus_shuffle = plus
shuffle(plus_shuffle)
#print(plus_shuffle[0])

minus_shuffle = []
minus_shuffle = minus
shuffle(minus_shuffle)
#print(type(int(minus_shuffle[0],2)))

train = plus_shuffle[:100] + minus_shuffle[:100]
test = plus_shuffle[101:201] + minus_shuffle[101:201]
remain = plus_shuffle[200:] + minus_shuffle[200:]
#print(type(int(train[0],2)))
#print(len(test))
#print(len(remain))
#print(plus)
#print(minus)

In [40]:
# Бинаризуем наши train, test контексты
train_bin = []
test_bin = []
for i in range(0,len(train)):
    train_bin.append(int(train[i][:-1],2))
    test_bin.append(int(test[i][:-1],2))
    
plus_bin = train_bin[:100]
minus_bin = train_bin[100:]

#print(len(plus_bin))
#print(len(minus_bin))

In [41]:
#Список голосов за каждый из тренировочных объектов
vote_list = []

#Функция фальсификации гипотез
def falsification(mask_bin, context_bin):
    for i in range(0,len(context_bin)-1):
        if mask_bin & context_bin[i] == mask_bin:
            return 0
        else:
            return 1

In [42]:
#Функция подсчета голосов для неизвестного объекта
def get_vote(to_detect, plus_bin, minus_bin):
    vote_count = 0
    for i in range(0,len(plus_bin)-1):
        mask = to_detect & plus_bin[i]
        vote_count += falsification(mask, minus_bin)
    for i in range(0,len(plus_bin)-1):
        mask = to_detect & minus_bin[i]
        vote_count -= falsification(mask, plus_bin)
    return vote_count

In [43]:
# Считаем количество голосов за каждый элемент из обучающей выборки

for i in range(0, len(train_bin)):
    to_detect = train_bin[i]
    vote = get_vote(to_detect, plus_bin, minus_bin)
    vote_list.append(vote)
    #print('vote for object #', i, '('+train[i]+')', vote)
    
cv_res = {
 "11": 0,  # true positive
 "10": 0,  # false negative
 "01": 0,  # false positive
 "00": 0,  # true negative
 "NaN": 0} # unclassified

In [44]:
# Считаем на оставшейся части
# Порог -20 получен эмпирически (ниже/выше дает хуже результаты)

for i in remain:
    det_str = i[:-1]
    to_detect = int(''.join(det_str))
    vote = get_vote(to_detect, plus_bin, minus_bin)
    if vote >= -20:
        decision = '1'
    elif vote < -20:
        decision = '0'
    else:
        decision = 'NaN'
    if i[0] == '1' and decision == '1':
        cv_res['11'] +=1
    elif i[0] == '1' and decision == '0':
        cv_res['10'] +=1
    elif i[0] == '0' and decision == '1':
        cv_res['01'] +=1
    elif i[0] == '0' and decision == '0':
        cv_res['00'] +=1
    else:
        cv_res['NaN'] +=1

print('Accuracy: ', (cv_res['11']+cv_res['00'])/(cv_res['11']+cv_res['00']+cv_res['01']+cv_res['10']+cv_res['NaN']))
print('True-positive: ', cv_res['11'], 'true-negative: ', cv_res['00'])
print('False-negative: ', cv_res['10'], 'false-positive: ', cv_res['01'])
print('Uncertain:', cv_res['NaN'])

Accuracy:  0.7522590361445783
True-positive:  995 true-negative:  4
False-negative:  18 false-positive:  311
Uncertain: 0


Возможно, что для повышения точности классификатора, нужно использовать более структурированный подход, например, используя минимальные гипотезы.